In [1]:
import json
import pandas as pd
import numpy as np

In [2]:
half = "H1"
### CHOOSE ONE
### H1, H2, None

In [3]:
with open("../matches/matches_England.json", "r") as file:
    match_data_2 = json.load(file)

df2 = pd.DataFrame(match_data_2)
if half == "H1":
    df2 = df2[df2.gameweek<20]
elif half == "H2":
    df2 = df2[df2.gameweek>=20]

with open("../matches/matches_France.json", "r") as file:
    match_data_3 = json.load(file)

df3 = pd.DataFrame(match_data_3)
if half == "H1":
    df3 = df3[df3.gameweek<20]
elif half == "H2":
    df3 = df3[df3.gameweek>=20]


with open("../matches/matches_Germany.json", "r") as file:
    match_data_4 = json.load(file)

df4 = pd.DataFrame(match_data_4)
if half == "H1":
    df4 = df4[df4.gameweek<18]
elif half == "H2":
    df4 = df4[df4.gameweek>=18]

with open("../matches/matches_Italy.json", "r") as file:
    match_data_5 = json.load(file)

df5 = pd.DataFrame(match_data_5)
if half == "H1":
    df5 = df5[df5.gameweek<20]
elif half == "H2":
    df5 = df5[df5.gameweek>=20]

with open("../matches/matches_Spain.json", "r") as file:
    match_data_6 = json.load(file)

df6 = pd.DataFrame(match_data_6)
if half == "H1":
    df6 = df6[df6.gameweek<20]
elif half == "H2":
    df6 = df6[df6.gameweek>=20]

all_matches_df = pd.concat([df2, df3, df4, df5, df6], ignore_index=True)
all_matches_list = list(np.unique(all_matches_df.wyId))

In [4]:
half = "H1"
### CHOOSE ONE
### H1, H2, None

with open("../matches/matches_England.json", "r") as file:
    match_data_2 = json.load(file)

df2 = pd.DataFrame(match_data_2)
if half == "H1":
    df2 = df2[df2.gameweek<20]
elif half == "H2":
    df2 = df2[df2.gameweek>=20]

with open("../matches/matches_France.json", "r") as file:
    match_data_3 = json.load(file)

df3 = pd.DataFrame(match_data_3)
if half == "H1":
    df3 = df3[df3.gameweek<20]
elif half == "H2":
    df3 = df3[df3.gameweek>=20]


with open("../matches/matches_Germany.json", "r") as file:
    match_data_4 = json.load(file)

df4 = pd.DataFrame(match_data_4)
if half == "H1":
    df4 = df4[df4.gameweek<18]
elif half == "H2":
    df4 = df4[df4.gameweek>=18]

with open("../matches/matches_Italy.json", "r") as file:
    match_data_5 = json.load(file)

df5 = pd.DataFrame(match_data_5)
if half == "H1":
    df5 = df5[df5.gameweek<20]
elif half == "H2":
    df5 = df5[df5.gameweek>=20]

with open("../matches/matches_Spain.json", "r") as file:
    match_data_6 = json.load(file)

df6 = pd.DataFrame(match_data_6)
if half == "H1":
    df6 = df6[df6.gameweek<20]
elif half == "H2":
    df6 = df6[df6.gameweek>=20]

all_matches_df = pd.concat([df2, df3, df4, df5, df6], ignore_index=True)

with open("../events/events_England.json", "r") as file:
    event_data_2 = json.load(file)

df2 = pd.DataFrame(event_data_2)
df2 = df2[df2.matchId.isin(all_matches_list)]

with open("../events/events_France.json", "r") as file:
    event_data_3 = json.load(file)

df3 = pd.DataFrame(event_data_3)
df3 = df3[df3.matchId.isin(all_matches_list)]


with open("../events/events_Germany.json", "r") as file:
    event_data_4 = json.load(file)

df4 = pd.DataFrame(event_data_4)
df4 = df4[df4.matchId.isin(all_matches_list)]


with open("../events/events_Italy.json", "r") as file:
    event_data_5 = json.load(file)

df5 = pd.DataFrame(event_data_5)
df5 = df5[df5.matchId.isin(all_matches_list)]


with open("../events/events_Spain.json", "r") as file:
    event_data_6 = json.load(file)

df6 = pd.DataFrame(event_data_6)
df6 = df6[df6.matchId.isin(all_matches_list)]

with open("../events/teams.json", "r") as file:
    teams_data = json.load(file)

teams_df = pd.DataFrame(teams_data)

In [5]:
all_events_df = pd.concat([df2, df3, df4, df5, df6], ignore_index=True)

# Feature engineering - Pass DF - Pass Motifs

In [6]:
passes_df = all_events_df[all_events_df['eventName'] == 'Pass']

# Concatenating teamId and matchId to create a unique identifier for each team in each match
passes_df['uniqueTeamId'] = passes_df['teamId'].astype(str) + "---" + passes_df['matchId'].astype(str)

<ipython-input-6-846069bd07cf>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  passes_df['uniqueTeamId'] = passes_df['teamId'].astype(str) + "---" + passes_df['matchId'].astype(str)


In [7]:
# Function to classify pass motifs
def classify_pass_motif(passes):
    players = passes['playerId'].values
    if len(set(players)) == 2:  # ABAB
        return 'ABAB'
    elif len(set(players)) == 3:
        if players[0] == players[2] or players[1] == players[3]:  # ABCA or ABCB
            return 'ABCA' if players[0] == players[2] else 'ABCB'
        else:
            return 'ABCD'  # ABCD for distinct players or other scenarios
    return 'ABCD'

# Apply the classification within a rolling window
def apply_motif_classification(df):
    # Ensure the DataFrame is sorted by event time
    df = df.sort_values(by='eventSec')
    motifs = []
    # Using a rolling window of 4 events
    for i in range(len(df) - 3):
        window = df.iloc[i:i+4]
        motif = classify_pass_motif(window)
        motifs.append((window.iloc[-1]['uniqueTeamId'], motif))
    return pd.DataFrame(motifs, columns=['uniqueTeamId', 'Motif'])

In [ ]:
motif_classification_results = passes_df.groupby('uniqueTeamId').apply(apply_motif_classification)

In [ ]:
motif_classification_results.reset_index(drop=True, inplace=True)

In [ ]:
motif_counts = motif_classification_results.groupby(['uniqueTeamId', 'Motif']).size().unstack(fill_value=0)

In [ ]:
motif_ratios = motif_counts.div(motif_counts.sum(axis=1), axis=0)
# df to be used in clustering

In [ ]:
motif_ratios

In [ ]:
motif_stats = motif_ratios.agg(['mean', 'std']).T

In [ ]:
motif_stats

# Feature engineering - Shot DF

In [ ]:
shot_df = all_events_df[all_events_df['eventName'] == 'Shot']
shot_df['uniqueTeamId'] = shot_df['teamId'].astype(str) + "---" + shot_df['matchId'].astype(str)

In [ ]:
import math

def calculate_shot_distance(row):
    if row['positions'] is not None and len(row['positions']) > 0:
        loc = row['positions'][0]
        # Normalize locations
        x = loc['x'] * (120 / 100.0)  # Normalizing the x coordinate
        y = loc['y'] * (80 / 100.0)   # Normalizing the y coordinate
        # Calculate Euclidean distance from (0,0)
        distance = math.sqrt((120-x)**2 + (80-y)**2)
        return distance
    return None

# Apply the function to calculate shot distance and add it to the DataFrame
shot_df['shot_distance'] = shot_df.apply(calculate_shot_distance, axis=1)

In [ ]:
def classify_shot_by_distance(distance):
    if distance <= 38:
        return "Near"
    elif distance <= 52:
        return "Middle Distance"
    else:
        return "Far"

# Assuming shot_df has a 'shot_distance' column calculated as before
shot_df['shot_class'] = shot_df['shot_distance'].apply(classify_shot_by_distance)

# Grouping shots by their class and counting
shot_class_counts = shot_df.groupby('uniqueTeamId')['shot_class'].value_counts().unstack(fill_value=0)
shot_counts = shot_df.groupby('uniqueTeamId')['subEventName'].value_counts().unstack(fill_value=0)

In [ ]:
shot_df_final = pd.concat([shot_counts, shot_class_counts], axis=1)

In [ ]:
shot_df_final

In [ ]:
shot_df_final.rename(columns = {'Shot':'Total Shots'}, inplace = True) 

shot_df_final["Far"] = shot_df_final["Far"]/shot_df_final["Total Shots"]
shot_df_final["Middle Distance"] = shot_df_final["Middle Distance"]/shot_df_final["Total Shots"]
shot_df_final["Near"] = shot_df_final["Near"]/shot_df_final["Total Shots"]


shot_df_final.rename(columns = {'Near':'Near Shots Ratio', 'Middle Distance':'Middle Distance Shots Ratio', 'Far':'Far Shots Ratio',}, inplace = True) 

In [ ]:
shot_df_final

# Feature engineering - Pass DF - Pass Types

In [ ]:
def determine_zone(position):
    if 'x' in position and 'y' in position:
        x, y = position['x'], position['y']
        zone = int(x // 33.333) + 1
        return zone
    else:
        return None
    
# Apply the function to determine zones for each event
passes_df['Zone'] = passes_df['positions'].apply(lambda pos: determine_zone(pos[0]) if len(pos) > 0 else None)

In [ ]:
passes_df.loc[passes_df['Zone'] == 4, ['Zone']] = 3

In [ ]:
passes_df

In [ ]:
import math

def calculate_pass_details_from_single_column(row):
    if row['positions'] is not None and len(row['positions']) == 2:
        # Extract and normalize start and end locations
        start_loc = row['positions'][0]
        end_loc = row['positions'][1]
        
        start_x = start_loc['x'] * (120 / 100.0)
        start_y = start_loc['y'] * (80 / 100.0)
        end_x = end_loc['x'] * (120 / 100.0)
        end_y = end_loc['y'] * (80 / 100.0)
        
        # Calculate pass length and angle
        delta_x = end_x - start_x
        delta_y = end_y - start_y
        pass_length = math.sqrt(delta_x**2 + delta_y**2)
        pass_angle = math.atan2(delta_y, delta_x)
        
        return pd.Series([pass_length, pass_angle])
    else:
        return pd.Series([None, None])

In [ ]:
passes_df[['calculated_pass_length', 'calculated_pass_angle']] = passes_df.apply(calculate_pass_details_from_single_column, axis=1)


In [ ]:
def determine_pass_direction(angle):
    # Forward Pass: -π/4 < angle < π/4
    if -np.pi/4 < angle < np.pi/4:
        return "Forward Pass"
    # Backward Pass: 3π/4 < angle <= π or -π <= angle < -3π/4
    elif 3*np.pi/4 < angle <= np.pi or -np.pi <= angle < -3*np.pi/4:
        return "Backward Pass"
    # Side Pass: Otherwise
    else:
        return "Side Pass"

In [ ]:
passes_df['pass_direction'] = passes_df['calculated_pass_angle'].apply(determine_pass_direction)

In [ ]:
passes_df

In [ ]:
np.unique(passes_df.subEventName)

In [ ]:
passes_df

In [ ]:
#passes_count = passes_df.groupby('teamId')['subEventName'].value_counts().unstack(fill_value=0)

In [ ]:
# Step 1: Combine 'pass_direction' and 'Zone' into a single column
passes_df['dir_zone'] = passes_df['pass_direction'].astype(str) + '_Zone_' + passes_df['Zone'].astype(str)

# Step 2: Group by 'uniqueTeamId' and the new 'dir_zone' column, then count
direction_zone_count = passes_df.groupby(['uniqueTeamId', 'dir_zone']).size()

# Step 3: Pivot the results to have 'uniqueTeamId' as rows and each unique 'dir_zone' combination as columns
direction_zone_count_pivot = direction_zone_count.unstack(fill_value=0)

direction_count = passes_df.groupby('uniqueTeamId')['pass_direction'].value_counts().unstack(fill_value=0)

pass_direction_df = pd.concat([direction_zone_count_pivot, direction_count], axis=1)

In [ ]:
pass_direction_df

## Pass Types

In [ ]:
high_pass_types = ['High pass', 'Cross', 'Launch']
# Define low pass types based on exclusion
all_pass_types = passes_df['subEventName'].unique()
low_pass_types = [pt for pt in all_pass_types if pt not in high_pass_types and pt != 'Smart pass']

# Group by 'uniqueTeamId' and count passes
pass_counts = passes_df.groupby(['uniqueTeamId', 'subEventName']).size().unstack(fill_value=0)
pass_counts['Total_Passes'] = pass_counts.sum(axis=1)

# Calculate high pass and low pass totals
pass_counts['High_pass_Total'] = pass_counts[high_pass_types].sum(axis=1)
pass_counts['Low_pass_Total'] = pass_counts[low_pass_types].sum(axis=1)

# Calculate overall high pass and low pass ratios
pass_counts['High_pass_Ratio_Total'] = pass_counts['High_pass_Total'] / pass_counts['Total_Passes']
pass_counts['Low_pass_Ratio_Total'] = pass_counts['Low_pass_Total'] / pass_counts['Total_Passes']

# Now, we'll prepare the final DataFrame with the required columns
final_columns = ['Total_Passes', 'High_pass_Ratio_Total', 'Low_pass_Ratio_Total']
final_pass_types_df = pass_counts[final_columns]

# If needed, calculate zone-specific ratios (demonstrated for High pass, replicate for Low pass as necessary)
zones = passes_df['Zone'].unique()
for zone in zones:
    zone_specific_passes = passes_df[passes_df['Zone'] == zone]
    zone_pass_counts = zone_specific_passes.groupby(['uniqueTeamId', 'subEventName']).size().unstack(fill_value=0)
    zone_pass_counts['Total_Passes'] = zone_pass_counts.sum(axis=1)
    
    # High pass total and ratio for the zone
    zone_pass_counts['High_pass_Total_Zone'] = zone_pass_counts[high_pass_types].sum(axis=1)
    final_pass_types_df[f'High_pass_Ratio_Zone{zone}'] = zone_pass_counts['High_pass_Total_Zone'] / zone_pass_counts['Total_Passes']
    
    # Low pass total and ratio for the zone
    zone_pass_counts['Low_pass_Total_Zone'] = zone_pass_counts[low_pass_types].sum(axis=1)
    final_pass_types_df[f'Low_pass_Ratio_Zone{zone}'] = zone_pass_counts['Low_pass_Total_Zone'] / zone_pass_counts['Total_Passes']

In [ ]:
final_pass_types_df

# Pass_Connectivity

In [ ]:
from collections import defaultdict
import networkx as nx

In [ ]:
def calculate_pass_connectivity(group):
    # Initialize two empty graphs for the two teams in the match
    team_graphs = {team_id: nx.Graph() for team_id in group['uniqueTeamId'].unique()}
    
    # Track the previous player and team to identify consecutive passes
    prev_player = None
    prev_team = None
    
    for i, pass_ in group.iterrows():
        current_player = pass_['playerId']
        current_team = pass_['uniqueTeamId']
        
        # Check for consecutive passes within the same team
        if prev_team == current_team and prev_player is not None:
            team_graphs[current_team].add_edge(prev_player, current_player)
        
        # Update the previous player and team for the next iteration
        prev_player = current_player
        prev_team = current_team
    
    # Calculate connectivity for each team graph
    connectivities = {}
    for team_id, graph in team_graphs.items():
        if len(graph) == 0:  # Handle cases where a team made no passes or only incomplete passes
            connectivity = 0
        else:
            connectivity = nx.average_node_connectivity(graph)
        connectivities[team_id] = connectivity
    
    return connectivities

In [ ]:
test_dfdf = passes_df.iloc[:1000]

In [ ]:
connectivity_results = [calculate_pass_connectivity(group) for _, group in passes_df.groupby(['matchId'])]

In [ ]:
connectivity_results

In [ ]:
connectivity_df = [(k, v) for d in connectivity_results for k, v in d.items()]
connectivity_df = pd.DataFrame(connectivity_df, columns=['uniqueTeamId', 'Connectivity'])

In [ ]:
connectivity_df.set_index('uniqueTeamId', inplace=True)
connectivity_df

In [ ]:
final_df = pd.concat([motif_ratios, final_pass_types_df, pass_direction_df, shot_df_final, connectivity_df], axis=1)
# Iterate through each column in final_df to find and adjust the specified columns
for col in final_df.columns:
    # Check if the column name contains 'Backward', 'Forward', or 'Side'
    if any(keyword in col for keyword in ['Backward', 'Forward', 'Side']):
        # Divide the column by the 'Total_Passes' column
        final_df[col] = final_df[col] / final_df['Total_Passes']
        # Rename the column to add " Ratio" at the end
        final_df.rename(columns={col: col + " Ratio"}, inplace=True)

In [47]:
final_df

,ABAB,ABCA,ABCB,ABCD,Total_Passes,High_pass_Ratio_Total,Low_pass_Ratio_Total,High_pass_Ratio_Zone2,Low_pass_Ratio_Zone2,High_pass_Ratio_Zone1,...,Side Pass_Zone_2 Ratio,Side Pass_Zone_3 Ratio,Backward Pass Ratio,Forward Pass Ratio,Side Pass Ratio,Total Shots,Far Shots Ratio,Middle Distance Shots Ratio,Near Shots Ratio,Connectivity
uniqueTeamId,,,,,,,,,,,,,,,,,,,,,
10531---2499726,0.039755,0.091743,0.088685,0.779817,330,0.127273,0.869697,0.055215,0.944785,0.198413,...,0.230303,0.081818,0.157576,0.339394,0.503030,4.0,0.250000,0.250000,0.500000,6.294872
10531---2499736,0.045570,0.106329,0.103797,0.744304,398,0.140704,0.844221,0.073864,0.909091,0.192000,...,0.246231,0.140704,0.155779,0.304020,0.540201,7.0,0.428571,0.571429,0.000000,6.659341
10531---2499741,0.022989,0.105747,0.105747,0.765517,438,0.146119,0.842466,0.130435,0.853755,0.197080,...,0.294521,0.043379,0.155251,0.349315,0.495434,6.0,0.166667,0.333333,0.500000,6.472527
10531---2499757,0.027344,0.103516,0.101562,0.767578,515,0.149515,0.829126,0.113744,0.867299,0.139535,...,0.205825,0.133981,0.163107,0.333981,0.502913,8.0,0.000000,0.625000,0.375000,7.153846
10531---2499766,0.032258,0.101382,0.105991,0.760369,220,0.240909,0.750000,0.134615,0.846154,0.351648,...,0.154545,0.077273,0.168182,0.450000,0.381818,3.0,0.000000,0.666667,0.333333,5.285714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
756---2565690,0.039267,0.096859,0.091623,0.772251,385,0.228571,0.766234,0.150538,0.844086,0.368852,...,0.225974,0.124675,0.171429,0.374026,0.454545,11.0,0.181818,0.545455,0.272727,7.679487
756---2565706,0.021552,0.120690,0.120690,0.737069,235,0.221277,0.765957,0.145833,0.833333,0.298077,...,0.191489,0.085106,0.157447,0.361702,0.480851,7.0,0.142857,0.714286,0.142857,5.636364
756---2565714,0.035088,0.081871,0.081871,0.801170,345,0.237681,0.753623,0.177143,0.811429,0.303704,...,0.257971,0.060870,0.130435,0.368116,0.501449,5.0,0.200000,0.400000,0.400000,8.296703


In [48]:
table_name = "final_in_poss_df"

if half is not None:
    final_df.to_csv(f'{table_name}_{half}.csv', index=True)
else:
    final_df.to_csv(f'{table_name}.csv', index=True)

# END OF FILE